In [1]:
# Data Science
import re
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
from pandasgui import show
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

In [ ]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\summary_stats_7_14.csv")

In [ ]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\MASTER_7_14.csv")

In [ ]:
matches

In [ ]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\keywords.csv")

In [ ]:
TOTAL_KEYWORD_MATCHES = list(total["Total_Count"].unique())[1]

In [ ]:
diverse_df = pd.DataFrame(columns = list(matches.columns))

In [ ]:
percent = 5000 / TOTAL_KEYWORD_MATCHES

int(math.ceil(24 * percent))

In [ ]:
keywords = ['dementia', 'cognition', 'memory', 'mmse', 'moca', 'alzheimer', 'cognitive impairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's",  'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia', 'ad', 'lbd']

In [ ]:
def diversity_sampling(SEQ_COUNT):
    # Constant declarations
    diverse_df = pd.DataFrame(columns = list(matches.columns))

    for word in (keywords):
        # getting subset of dataframe that contains sequence with keyword match
        contains = matches['padded_regex_match'].str.contains(str(word))
        filtered = matches[contains]
        subset = pd.DataFrame(columns = list(matches.columns))  
        # calculating percent that will be taken from each set of keyword matches
        percent = SEQ_COUNT / TOTAL_KEYWORD_MATCHES
        # getting number of matches to take from subset of keyword matches
        take = int(math.ceil(len(filtered) * percent))
        
        if (take == 1):
            take = 2
            
        print("Word:", word, "Take: ", take , "EMPI: ", len(filtered["EMPI"]), "Unique EMPI: ", len(filtered["EMPI"].unique()))

        if (take <= len(filtered["EMPI"].unique())):
            # priortizing getting sequences from unique patients if size of take permits
            filtered = filtered.drop_duplicates(subset = "EMPI", keep = "first")
            
            unique_empi = diverse_df["EMPI"].unique().tolist()
            
            filtered["in_diverse_df"] = np.where(filtered["EMPI"].isin(unique_empi), 'Y', 'N')
            new_additions = filtered[filtered["in_diverse_df"] == 'N']
            new_additions = pd.DataFrame(new_additions, columns = list(matches.columns))
            
#             print("New Additions: ", len(new_additions))
            idx = 0
            if take <= len(new_additions):
                idx = take
            else:
                idx = new_additions - 1
            new_additions = new_additions.iloc[:idx]
            subset = subset.append(new_additions)
            print("Subset After New Additions: ", len(subset), "Subset Unique EMPI: ", len(subset["EMPI"].unique().tolist()))
                    
            take -= len(new_additions)
            
            if (take > 0):
                filtered.drop(new_additions.index, axis = 0, inplace = True)

                # generating random index numbers for subset
                for _ in range(take):
                    idx = random.sample(range(0, len(filtered)), take)
                    subset = subset.append(pd.DataFrame(filtered.iloc[idx]).T)

                print("Subset: ", len(subset))
        else:
            # getting as many unique patient records as possible even if take > len(filtered["EMPI"].unique())
            partial_subset = filtered.drop_duplicates(subset = "EMPI", keep = "first")
            print("PS: ", len(partial_subset))
            subset.append(partial_subset)
            take -= len(partial_subset)
            filtered.drop(partial_subset.index, axis = 0, inplace = True)

            # generating random index numbers for subset
            for _ in range(take):
                idx = random.sample(range(0, len(filtered)), take)
                subset = subset.append(pd.DataFrame(matches.iloc[idx]).T)
                #print(len(subset))  

        diverse_df = diverse_df.append(subset)
        print("Unique EMPI: ", len(diverse_df["EMPI"].unique()), "Len: ", len(diverse_df), "\n")
    return diverse_df

In [ ]:
diverse_df = diversity_sampling(5000)

In [ ]:
diverse_df = diverse_df.reset_index()
diverse_df

In [ ]:
len(diverse_df["PatientID"].unique())

In [ ]:
diverse_df.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_sample_7_19.csv")

## Checking for Keyword Diversity

In [ ]:
# defining summary stats df
cols =  ['dementia', 'cognition', 'memory', 'mmse', 'moca', 'alzheimer', 'cognitive impairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's",  'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia', 'ad', 'lbd']

summary_stats_seq  = pd.DataFrame(pd.np.empty((len(diverse_df), len(cols) + 3)) * pd.np.nan) 
summary_stats_seq.columns = ['EMPI', 'EncounterID', 'NoteID'] + list(cols)
summary_stats_seq["EMPI"] = diverse_df["EMPI"].to_list()
summary_stats_seq["EncounterID"] = diverse_df["PatientEncounterID"].to_list()
summary_stats_seq["NoteID"]  = diverse_df["NoteID"].to_list()

In [ ]:
for i in tqdm(range(len(d))):
    match_tuples = []
    keyword_list = []
    

    keyword_list = (eval(d['regex_match'][i]))
    keyword_list = list(eval(keyword_list))
    for z in range(len(keyword_list)):
        match_tuples.append((keyword_list[z], 0))

    freq = {"dementia":0
        ,"cognition":0
        ,"cognition":0
        ,"memory":0
        ,"mmse":0
        ,"moca":0
        ,"alzheimer":0
        ,'cognitive impairment':0
        ,"mci":0
        ,"cerebellar":0
        ,"neurocognitive":0
        ,"lewy":0
        ,"pick's":0
        ,"corticobasal":0
        ,"cerebral":0
        ,"cerebrovascular":0
        ,"amnesia":0
        ,"ad": 0
        ,"lbd": 0
    }

    for k, v in match_tuples:   
        if (k != ''):
            if (k == 'cognitive impairmentcognitiveimpairment' or k == 'cognitiveimpairmentcognitive impairment'):
                k = 'cognitive impairment'
                freq.update({str(k.lower()):int(freq[k.lower()] + 1)})
            else:
                freq.update({str(k.lower()):int(freq[k.lower()] + 1)})
            
    #print(freq.items())
    #print(summary_stats["dementia"])
    for k, v in freq.items():
        #print(str(k.lower()))
        summary_stats_seq[str(k.lower())][i] = int(v)

In [ ]:
summary_stats_seq.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_seqs_7_23.csv")

## Executing SLAT Pipeline Preprocessing

In [38]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\SLAT_production_7_22.csv")

## Re-running keyword matches

In [22]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\keywords.csv")

In [23]:
k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == 0):
        p_list.append(re.compile(k[i][5:], re.IGNORECASE))
    elif (c[i] == 1):
        p_list.append(re.compile(k[i]))
print(p_list)

[re.compile('\\bdementia\\b', re.IGNORECASE), re.compile('\\bcognition\\b', re.IGNORECASE), re.compile('\\bmemory\\b', re.IGNORECASE), re.compile('\\bMMSE\\b', re.IGNORECASE), re.compile('\\bMOCA\\b', re.IGNORECASE), re.compile('\\bAlzheimer\\b', re.IGNORECASE), re.compile('\\bcognitive\\s*impairment\\b', re.IGNORECASE), re.compile('(?-i:)\\bMCI\\b'), re.compile('\\bcerebellar\\b', re.IGNORECASE), re.compile('\\bNeurocognitive\\b', re.IGNORECASE), re.compile('\\blewy\\b', re.IGNORECASE), re.compile("\\bpick's\\b", re.IGNORECASE), re.compile('\\bCorticobasal\\b', re.IGNORECASE), re.compile('\\bcerebral\\b', re.IGNORECASE), re.compile('\\bcerebrovascular\\b', re.IGNORECASE), re.compile('\\bamnesia\\b', re.IGNORECASE), re.compile('(?-i:)\\bAD\\b'), re.compile('(?-i:)\\bLBD\\b')]


In [24]:
locations = []

for r in tqdm(matches['regex_sent']):
    l = []
    for p in (p_list):
        for match in re.finditer(p, r):
            l.append(match.span())
    #print(l)
    locations.append(l)
        
matches["regex_location"] = locations

100%|████████████████████████████████████████████████████████████████████████| 279224/279224 [00:43<00:00, 6356.01it/s]


In [41]:
l = []
for note in tqdm(matches["regex_sent"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set((m)))
        if (m != []):
            curr.append("".join(m))
    #print(curr)
    #print(l)
    l.append((curr))

matches["regex_match"] = l

100%|████████████████████████████████████████████████████████████████████████| 279224/279224 [00:44<00:00, 6254.89it/s]


In [42]:
matches

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6352398,10040002098,100000272,3.242118e+09,Progress Notes,2370049295,2.338373e+09,2019-03-14,----------------------------------------------...,"""ession alone in the meta-analysis. We discuss...",[dementia]
1,Z6352922,200766,100000894,3.181499e+09,Progress Notes,1916263837,1.871982e+09,2018-03-28,----------------------------------------------...,""" ------- sis over her R bra strap, which was ...","[dementia, Memory, MOCA]"
2,Z6353136,10027033512,100001150,3.018789e+09,Discharge Summary,267681869,2.555255e+08,2015-01-09,----------------------------------------------...,""" ------- l obstruction ASSOCIATED DIAGNOSES S...","[Dementia, cognitive impairment]"
3,Z6353461,10040011735,100001521,3.172504e+09,Progress Notes,1682366001,1.631850e+09,2017-10-03,----------------------------------------------...,""" ------- (156 lb) 09/19/17 71.4 kg (157 lb 6...","[dementia, Cognition]"
4,Z6353764,10040014499,100001873,3.165983e+09,Progress Notes,1853759390,1.807703e+09,2018-02-07,----------------------------------------------...,"""ut answer. Optho notes from recent outpatient...",[dementia]
...,...,...,...,...,...,...,...,...,...,...,...
279219,Z16320629,10127549029,113705759,3.285425e+09,Progress Notes,3960511043,3.943607e+09,2020-04-28,----------------------------------------------...,"""''she took a break' and wants to discuss with...",[memory]
279220,Z16320629,10127549029,113705759,3.305416e+09,Progress Notes,4585859437,4.574134e+09,2020-09-22,----------------------------------------------...,"""'all of her medications, including aspirin. D...",[memory]
279221,Z16320629,10127549029,113705759,3.327430e+09,Progress Notes,5203067949,5.224024e+09,2021-01-25,----------------------------------------------...,"""'Patient, Chart, daughter Olga Mode of contac...",[cognition]
279222,Z16320629,10127549029,113705759,3.349755e+09,Progress Notes,5673786544,5.670392e+09,2021-04-20,----------------------------------------------...,"""'Patient, Chart, daughter Olga Mode of contac...",[cognition]


In [50]:
(matches["regex_match"].iloc[[279000]])

279000    [Dementia, cognitive impairment, Lewy]
Name: regex_match, dtype: object

In [51]:
matches["ContactDTS"] = pd.to_datetime(matches["ContactDTS"]).dt.strftime('%Y-%m-%d')

In [52]:
matches['regex_sent'] = matches['regex_sent'].apply(json.dumps)
matches['regex_match'] = matches['regex_match'].apply(json.dumps)

In [35]:
matches = matches.reset_index()

In [36]:
matches

,index,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match,regex_location
0,0,Z6352398,10040002098,100000272,3.242118e+09,Progress Notes,2370049295,2.338373e+09,2019-03-14,----------------------------------------------...,"""\""ession alone in the meta-analysis. We discu...","""['dementia']""","[(397, 405)]"
1,1,Z6352922,200766,100000894,3.181499e+09,Progress Notes,1916263837,1.871982e+09,2018-03-28,----------------------------------------------...,"""\"" ------- sis over her R bra strap, which wa...","""['dementia', 'memory', 'moca']""","[(256, 264), (561, 569), (217, 223), (513, 519..."
2,2,Z6353136,10027033512,100001150,3.018789e+09,Discharge Summary,267681869,2.555255e+08,2015-01-09,----------------------------------------------...,"""\"" ------- l obstruction ASSOCIATED DIAGNOSES...","""['dementia', 'cognitive impairment']""","[(180, 188), (627, 635), (398, 418)]"
3,3,Z6353461,10040011735,100001521,3.172504e+09,Progress Notes,1682366001,1.631850e+09,2017-10-03,----------------------------------------------...,"""\"" ------- (156 lb) 09/19/17 71.4 kg (157 lb...","""['dementia', 'cognition']""","[(512, 520), (294, 303)]"
4,4,Z6353764,10040014499,100001873,3.165983e+09,Progress Notes,1853759390,1.807703e+09,2018-02-07,----------------------------------------------...,"""\""ut answer. Optho notes from recent outpatie...","""['dementia']""","[(395, 403)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
279219,279219,Z16320629,10127549029,113705759,3.285425e+09,Progress Notes,3960511043,3.943607e+09,2020-04-28,----------------------------------------------...,"""\""''she took a break' and wants to discuss wi...","""['memory']""","[(99, 105)]"
279220,279220,Z16320629,10127549029,113705759,3.305416e+09,Progress Notes,4585859437,4.574134e+09,2020-09-22,----------------------------------------------...,"""\""'all of her medications, including aspirin....","""['memory']""","[(101, 107)]"
279221,279221,Z16320629,10127549029,113705759,3.327430e+09,Progress Notes,5203067949,5.224024e+09,2021-01-25,----------------------------------------------...,"""\""'Patient, Chart, daughter Olga Mode of cont...","""['cognition']""","[(100, 109)]"
279222,279222,Z16320629,10127549029,113705759,3.349755e+09,Progress Notes,5673786544,5.670392e+09,2021-04-20,----------------------------------------------...,"""\""'Patient, Chart, daughter Olga Mode of cont...","""['cognition']""","[(100, 109)]"


In [ ]:
# locations = []

# for i in tqdm(range(len(merged['regex_sent']))):
#     r = merged['regex_sent'][i]
#     l = []
#     for p in p_list:
#         for match in re.finditer(p, r):
#             l.append(match.group().lower())
#             l = list(set(l))
#     #print(l)
#     merged.append(l)
        
# merged["regex_match"] = locations


In [ ]:
l = []
for note in tqdm(merged["regex_sent"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
    #print(curr)
    #print(l)
    l.append(str(curr))

merged["sequence_level_regex_match"] = l

In [53]:
matches[:1000].to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\regex_match_test.csv", index = False)

## Truncating notes to max 131K characters

In [60]:
matches["NoteTXT"] = matches["NoteTXT"].str[:131000]

In [63]:
matches["len"]=matches["NoteTXT"].str.len()
matches["len"].max()

131000

In [64]:
matches = matches[['PatientID', 'MRN', 'EMPI', 'PatientEncounterID',
       'InpatientNoteTypeDSC', 'NoteID', 'NoteCSNID', 'ContactDTS', 'NoteTXT',
       'regex_sent', 'regex_match']]

In [65]:
matches.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\SLAT_production_7_24.csv", index = False)

In [ ]:
production.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\SLAT_production_7_22.csv", index = False)